In [1]:
import glob
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
import IPython

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, Conv2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

import os

# Any results you write to the current directory are saved as output.
from google.colab import drive
drive.mount('/content/gdrive')

Using TensorFlow backend.


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
print("The current directory is: ", os.getcwd())
import os
os.chdir("/content/gdrive/My Drive/Galvanize Adm/TF_Speech_Recognition")
print("The current directory is: ", os.getcwd())

The current directory is:  /content
The current directory is:  /content/gdrive/My Drive/Galvanize Adm/TF_Speech_Recognition


In [3]:
os.listdir()

['TF_Speech_Recognition.ipynb',
 'dfff.csv',
 'val_list.txt',
 'test_list.txt',
 'array.csv']

In [0]:
# for use in Kaggle as the audio files are not saved in Colabs - Loading the files, let's see total audio files for each label
# label = []
# counts = []
# for i in os.listdir('../input/train/audio/'):
#     label.append(i)
#     counts.append(len(os.listdir('../input/train/audio/' + i)))
    
# df = pd.DataFrame([label, counts]).T.set_index([0])
# df

In [0]:
# # Total number of audio files
# np.sum(df[1])

In [0]:
# os.listdir('../input/')

In [0]:
# Saving files from test and validation list
# test_list = open('../input/train/testing_list.txt').read().split('\n')
# val_list = open('../input/train/validation_list.txt').read().split('\n')

In [0]:
# # let us save the above list in our directory, so we can load it afterwards.  This will be easier for us to avoid opening the aduio files
# with open("test_list.txt", "w") as output:
#     output.write(str(test_list))
    
# with open("val_list.txt", "w") as output:
#     output.write(str(val_list))

In [0]:
# Checking the lenght of the list
# len(test_list), len(val_list)

In [0]:
#Playint on audio file
# IPython.display.Audio('../input/train/audio/right/bf8d5617_nohash_0.wav')

In [0]:
#Labels that Kaggle has asked us to use for the competition
# labels_in_test = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
# df = df.loc[labels_in_test]
# df

In [0]:
# Total number of files in the labels_in_test
# np.sum(df[1])

In [0]:
# Loading the audio file as an array using the librosa's mfcc - we do this since our keras model only understands arrays(or numbers) and we
# cannot feed the audio file directly.
# %%time

# def extract_data(folder):
#     dff = pd.DataFrame()
#     names = []
#     features = []
    
#     for name in os.listdir('../input/train/audio/' + folder):
#         data, sampling_rate = librosa.load('../input/train/audio/' + folder + '/' + name, res_type = 'kaiser_fast')
#         mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sampling_rate, n_mfcc = 40).T, axis = 0)
        
#         features.append(mfccs)
#         names.append(name)
        
#     return [names, features]   # the function is written in the names of the files and audio arrays

# train_new = pd.Series(labels_in_test).apply(extract_data)   #apply the function for the items in labels_in_test only so that it can 
                                                              #can make a new DataFrame with the names and features of the audio files
# train_new.columns = ['names', 'features']

In [0]:
# Connecting all the lists into one DataFrame
# dfff = pd.DataFrame()
# dfff['names'] = train_new[0][0] + train_new[1][0] + train_new[2][0] + train_new[3][0] + train_new[4][0] + train_new[5][0] + train_new[6][0] + train_new[7][0] + train_new[8][0] + train_new[9][0]
# dfff['feature'] = train_new[0][1] + train_new[1][1] + train_new[2][1] + train_new[3][1] + train_new[4][1] + train_new[5][1] + train_new[6][1] + train_new[7][1] + train_new[8][1] + train_new[9][1]

In [0]:
# Making a new list for the label column
# label_all = []
# for index in labels_in_test:
#     label_all.append([index] * df.loc[index][1])

In [0]:
# new list for the label column is now being put into the DataFrame
# dfff['label'] = [item for sublist in label_all for item in sublist]

In [0]:
# print(dfff.shape)
# dfff.head()

In [0]:
# Saving the DataFrame as a csv file to avoid using the audio files going forward
# dfff.to_csv("dfff.csv", index = False, sep = ',')

In [0]:
# Converting the string of arrays to a proper array
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')

In [0]:
dfff = pd.read_csv("dfff.csv", converters = {'feature' : converter})

In [6]:
dfff.head()

,Unnamed: 0,names,feature,label
0,0,5fe4a278_nohash_1.wav,"[-360.960392, 95.2412528, -45.965287, 61.02619...",yes
1,1,4def68db_nohash_1.wav,"[-350.560308, 95.8547696, -18.2142555, 43.4533...",yes
2,2,3d53244b_nohash_3.wav,"[-452.935767, 48.5914362, -8.35673584, 39.4135...",yes
3,3,62ef962d_nohash_1.wav,"[-304.877085, 60.8177152, -32.5946494, 18.1406...",yes
4,4,264f471d_nohash_3.wav,"[-305.40941638, 100.41015856, -2.31436065, 58....",yes


In [0]:
# Loading the test and validation list
import ast
with open("test_list.txt", "r") as f:
    test_list = ast.literal_eval(f.read())
    
with open("val_list.txt", "r") as f:
    val_list = ast.literal_eval(f.read())

In [8]:
# Extracting the names of the files which Kaggle has asked us to validate upon
val_list_all = pd.Series(val_list[:-1]).apply(lambda x: x.split('/')[1])
test_list_all = pd.Series(test_list[:-1]).apply(lambda x: x.split('/')[1])
len(val_list_all), len(test_list_all)

(6798, 6835)

In [9]:
# Making a DataFrame from the val_list
val = dfff[dfff.names.isin(val_list_all)]
val = val.iloc[np.random.permutation(len(val))]  # Shuffle DataFrame rows
print(val.shape)

train = dfff[~dfff.names.isin(val_list_all)]
train = train.iloc[np.random.permutation(len(train))] # Shuffle DataFrame rows
print(train.shape)

(2577, 4)
(21105, 4)


In [10]:
train.head()

,Unnamed: 0,names,feature,label
16269,16269,de41f895_nohash_2.wav,"[-260.072402, 130.215152, -2.64483555, 8.99604...",on
10702,10702,3e31dffe_nohash_3.wav,"[-447.718219, 114.64052, -10.2324393, 18.95816...",left
19222,19222,c22d3f18_nohash_1.wav,"[-336.906291, 83.9405475, -25.4507612, 52.6283...",stop
11088,11088,c6ca5d01_nohash_0.wav,"[-255.230781, 121.728997, -12.8546624, 59.6240...",left
10384,10384,3c8dc702_nohash_2.wav,"[-366.14613, 55.0088984, -14.4965602, 15.91847...",left


In [0]:
#Splitting into our X and y array
X_train = np.array(train.feature.tolist())
y_train = np.array(train.label.tolist())

X_val = np.array(val.feature.tolist())
y_val = np.array(val.label.tolist())

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_val = np_utils.to_categorical(lb.fit_transform(y_val))

In [12]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((21105, 40), (21105, 10), (2577, 40), (2577, 10))

In [13]:
# Here we are using deep learning using a sequential model with three hidden layers
ncols = X_train.shape[1]
model = Sequential()

model.add(Dense(1024, activation = 'relu', input_shape = (ncols,)))
model.add(Dropout(0.5))

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(y_train.shape[1], activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [131]:
# Fitting the model
%%time
model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = [X_val, y_val])

Instructions for updating:
Use tf.cast instead.
Train on 21105 samples, validate on 2577 samples
Epoch 1/10
21105/21105 [==============================] - 12s 554us/step - loss: 14.4258 - acc: 0.1020 - val_loss: 14.5795 - val_acc: 0.0955
Epoch 2/10
21105/21105 [==============================] - 11s 502us/step - loss: 14.4946 - acc: 0.1007 - val_loss: 14.5795 - val_acc: 0.0955
Epoch 3/10
21105/21105 [==============================] - 11s 540us/step - loss: 14.4890 - acc: 0.1011 - val_loss: 14.5795 - val_acc: 0.0955
Epoch 4/10
21105/21105 [==============================] - 11s 517us/step - loss: 14.4906 - acc: 0.1010 - val_loss: 14.5795 - val_acc: 0.0955
Epoch 5/10
21105/21105 [==============================] - 11s 518us/step - loss: 14.4899 - acc: 0.1010 - val_loss: 14.5795 - val_acc: 0.0955
Epoch 6/10
21105/21105 [==============================] - 11s 521us/step - loss: 14.4883 - acc: 0.1011 - val_loss: 14.5795 - val_acc: 0.0955
Epoch 7/10
21105/21105 [==============================] -

In [0]:
# Applying CNN (deep learning)
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(8, 5, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 7, 4, 32)          160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 2, 32)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 3, 2, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               24704     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total para

In [16]:
# Applying CNN (deep learning)
%%time
model.fit(X_train.reshape(X_train.shape[0], 8, 5, 1), y_train, batch_size = 32, epochs = 50, validation_data = [X_val.reshape(X_val.shape[0], 8, 5, 1), y_val], verbose = 1)

Instructions for updating:
Use tf.cast instead.
Train on 21105 samples, validate on 2577 samples
Epoch 1/50
21105/21105 [==============================] - 7s 314us/step - loss: 3.6054 - acc: 0.1504 - val_loss: 2.0993 - val_acc: 0.2142
Epoch 2/50
21105/21105 [==============================] - 4s 174us/step - loss: 2.1374 - acc: 0.2044 - val_loss: 1.9967 - val_acc: 0.2794
Epoch 3/50
21105/21105 [==============================] - 4s 177us/step - loss: 2.0645 - acc: 0.2359 - val_loss: 1.9336 - val_acc: 0.3217
Epoch 4/50
21105/21105 [==============================] - 4s 178us/step - loss: 2.0237 - acc: 0.2606 - val_loss: 1.8970 - val_acc: 0.3279
Epoch 5/50
21105/21105 [==============================] - 4s 176us/step - loss: 2.0019 - acc: 0.2682 - val_loss: 1.8774 - val_acc: 0.3291
Epoch 6/50
21105/21105 [==============================] - 4s 176us/step - loss: 1.9883 - acc: 0.2737 - val_loss: 1.8833 - val_acc: 0.3380
Epoch 7/50
21105/21105 [==============================] - 4s 175us/step - l

In [0]:
# Applying CNN with 4 hidden layers, out of which 1 is a CNN layer and 3 are dense layers (deep learning)
model = Sequential()
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', input_shape=(8, 5, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (2, 2), padding="same", activation="relu", input_shape=(8, 5, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
          
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
          
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(y_train.shape[1], activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 7, 4, 64)          320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 2, 64)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 3, 2, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 2, 32)          8224      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1, 1, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
__________

In [20]:
# Fitting the model 
%%time
model.fit(X_train.reshape(X_train.shape[0], 8, 5, 1), y_train, batch_size = 32, epochs = 50, validation_data = [X_val.reshape(X_val.shape[0], 8, 5, 1), y_val], verbose = 1)

Train on 21105 samples, validate on 2577 samples
Epoch 1/50
21105/21105 [==============================] - 5s 247us/step - loss: 2.1378 - acc: 0.2087 - val_loss: 2.0859 - val_acc: 0.2390
Epoch 2/50
21105/21105 [==============================] - 5s 243us/step - loss: 2.1378 - acc: 0.2096 - val_loss: 2.0807 - val_acc: 0.2507
Epoch 3/50
21105/21105 [==============================] - 5s 244us/step - loss: 2.1299 - acc: 0.2166 - val_loss: 2.0796 - val_acc: 0.2549
Epoch 4/50
21105/21105 [==============================] - 5s 246us/step - loss: 2.1246 - acc: 0.2149 - val_loss: 2.0836 - val_acc: 0.2375
Epoch 5/50
21105/21105 [==============================] - 5s 247us/step - loss: 2.1242 - acc: 0.2208 - val_loss: 2.0580 - val_acc: 0.2627
Epoch 6/50
21105/21105 [==============================] - 5s 246us/step - loss: 2.1189 - acc: 0.2181 - val_loss: 2.0549 - val_acc: 0.2565
Epoch 7/50
21105/21105 [==============================] - 5s 247us/step - loss: 2.1151 - acc: 0.2268 - val_loss: 2.0585 - v